# Task 1

Result of Task 1 can be found by running `main.py` and accessing the API through http://127.0.0.1:8000/api/v1/rec/user/{user_id}.

# Task 2

Result of Task 2.

In [1]:
import pandas as pd
import numpy as np

from _load_data import *
from _engineer_features import *

In [2]:
df_user, df_menu, df_sales, df_bridge = load_data()
df_user, df_menu, df_sales, df_bridge, df_rating = engineer_features(
    df_user, df_menu, df_sales, df_bridge
    )

In [26]:
df_menu

,menu_id,concept,brand,menu_category,menu_category_detail,menu_name,last_sold,avg_sales_interval
0,5204e17e-aefb-4235-8eb8-7177a5679352,Lifestyle,A/A,FOOD,APPETIZER,french fries,14 days 04:41:53,71 days 22:51:21
1,f026874c-5a66-4184-8ce0-9aeb9e37b5ac,Lifestyle,A/A,LIQUEUR,WHISK(E)Y,monkey shoulder_double,86 days 03:33:14,16 days 18:16:30.028860028
2,cfa23fc2-c4ab-4377-b0fe-489493016af1,Lifestyle,A/A,LIQUEUR,WHISK(E)Y,lagavulin 16 yo (glass),86 days 03:33:14,16 days 18:16:30.028860028
3,9708a976-0852-4705-a619-359447db7b09,Lifestyle,A/A,LIQUEUR,COCKTAIL,lemon drop,86 days 03:33:14,16 days 18:16:30.028860028
4,5aca9660-a702-4590-82dd-4c3666204fb8,Lifestyle,A/A,LIQUEUR,COCKTAIL,ginger,55 days 03:02:34,5 days 04:05:06.666666666
...,...,...,...,...,...,...,...,...
2010,85d91c3d-f2bb-4a1f-8cc4-b84352fc153a,Resto,Djournal House,FOOD,SOUP & SALAD,chicken sesame salad,54 days 06:26:21,6 days 23:26:18.800000
2011,d514f3e5-f9b1-46ff-9b47-fce239798e9a,Resto,Djournal House,BEVERAGE,WATER & SOFT DRINKS,aqua sparkling,89 days 03:37:55,16 days 07:02:24.376666666
2012,b49d2d3c-5182-4e7d-a54b-381a79e49eb6,Resto,Djournal House,BEVERAGE,TEA & COFFEE,earl grey,45 days 14:44:49,16 days 07:02:24.376666666
2013,78c8bb3b-d54c-474b-b04b-d66f42b9036e,Resto,Djournal House,BEVERAGE,TEA & COFFEE,hot premium tea,45 days 14:44:49,16 days 07:02:24.376666666


In [69]:
def make_func(brand):
    return lambda x: np.sum(x == brand)

brand_aggregator = {f"count_brand_{brand}": ('brand', make_func(brand)) for brand in df_menu['brand'].unique()}
brand_aggregator

{'count_brand_A/A': ('brand',
  <function __main__.make_func.<locals>.<lambda>(x)>),
 'count_brand_Djournal': ('brand',
  <function __main__.make_func.<locals>.<lambda>(x)>),
 'count_brand_Osteria GIA': ('brand',
  <function __main__.make_func.<locals>.<lambda>(x)>),
 'count_brand_Publik Markette': ('brand',
  <function __main__.make_func.<locals>.<lambda>(x)>),
 'count_brand_Pizza E Birra': ('brand',
  <function __main__.make_func.<locals>.<lambda>(x)>),
 'count_brand_DF Kitchen': ('brand',
  <function __main__.make_func.<locals>.<lambda>(x)>),
 'count_brand_Mr Fox': ('brand',
  <function __main__.make_func.<locals>.<lambda>(x)>),
 'count_brand_Sushi Groove': ('brand',
  <function __main__.make_func.<locals>.<lambda>(x)>),
 'count_brand_Social Garden': ('brand',
  <function __main__.make_func.<locals>.<lambda>(x)>),
 'count_brand_Skye': ('brand',
  <function __main__.make_func.<locals>.<lambda>(x)>),
 'count_brand_TPC': ('brand',
  <function __main__.make_func.<locals>.<lambda>(x)>),


In [70]:
(df_bridge
 .merge(df_sales, on='sales_id')
 .merge(df_menu, on='menu_id')
 .groupby('user_id')
 .agg(**brand_aggregator)
#  .droplevel(0, axis=1)
 )

,count_brand_A/A,count_brand_Djournal,count_brand_Osteria GIA,count_brand_Publik Markette,count_brand_Pizza E Birra,count_brand_DF Kitchen,count_brand_Mr Fox,count_brand_Sushi Groove,count_brand_Social Garden,count_brand_Skye,count_brand_TPC,count_brand_Social House,count_brand_Tokyo Belly,count_brand_Kitchenette,count_brand_Manarai,count_brand_Djournal House
user_id,,,,,,,,,,,,,,,,
00b158ed-0fcb-48e1-b627-2bb572a344cf,0,40,0,0,0,0,0,0,0,0,21,0,0,0,0,0
00bafc5b-3e97-439c-8b1f-9a6568adbaaa,0,0,5,0,4,0,0,0,0,0,0,5,0,0,0,0
00cf8833-2cd5-40df-8477-3e9b73dbdb66,0,0,9,0,0,0,0,0,0,0,0,0,0,4,0,0
0120f7c6-b3e7-4ff3-9082-90d311ca16f7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,0
014fc003-3f5b-4294-b2e6-836a17a0ed79,0,3,0,0,0,0,0,0,0,0,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fe476c7f-6392-43b4-887a-602372a7dfce,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0
fe8b2cb6-180e-443b-bb69-d44038034bd9,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0
ff13b265-54d4-48c4-8b51-f2e29e024e54,0,29,0,0,0,0,0,0,0,0,0,0,0,5,0,0


Insight 1: The higher the tier, the longer they have been member and the shorter their transaction interval.

In [5]:
(df_user
 .groupby(['user_tier_level', 'user_gender'])
 .agg({'time_since_created': 'mean',
       'avg_trx_interval': 'mean'})
 
 )

time_since_created  \
user_tier_level user_gender                               
Basic           F           410 days 12:06:06.894736844   
                M           410 days 23:51:19.854922280   
Silver          F           466 days 11:59:21.844262288   
                M           481 days 00:24:40.921568624   
Gold            F           514 days 06:50:05.424242424   
                M           610 days 07:29:41.238095240   
Diamond         F           540 days 07:53:34.041666664   
                M           502 days 05:05:09.923076920   
Black           F                     730 days 15:18:11   
                M           732 days 10:52:52.333333336   

                                      avg_trx_interval  
user_tier_level user_gender                             
Basic           F           25 days 17:11:52.681799490  
                M           26 days 10:20:46.549568439  
Silver          F           22 days 17:18:38.618533763  
                M           20 days 23:22:01.297522517  
Gold            F           16 days 16:02:35.173082028  
                M           23 days 06:22:46.117830025  
Diamond         F           12 days 13:17:00.893463244  
                M            8 days 11:27:57.925316467  
Black           F            2 days 08:34:47.451612903  
                M            6 days 09:09:24.968542568